[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/geo-di-lab/emerge-lessons/blob/main/docs/ch6/lesson3.ipynb)

# Earth Engine Data
This template file shows how you can download climate and environmental data from a point & time using Google Earth Engine.

Note: If you do not already have Google Earth Engine set up, [this lesson](https://geo-di-lab.github.io/emerge-lessons/docs/ch1/lesson5a.html) explains how to sign up and get access! Google Earth Engine is free to use for research, education, and nonprofit use.

In [1]:
import folium
import ee
import geemap
import geopandas as gpd
ee.Authenticate()

# Write your project ID here, in quotes
ee.Initialize(project = "emerge-lessons")

def add_ee_layer(self, ee_image_object, vis_params, name):
    """Adds a method for displaying Earth Engine image tiles to folium map."""
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)

folium.Map.add_ee_layer = add_ee_layer

## Load Earth Engine Data

Go to the [Earth Engine Data Catalog](https://developers.google.com/earth-engine/datasets/catalog) and find the dataset(s) you are interested in. There are a wide variety of datasets available on Google Earth Engine, including temperature, land cover, satellite, and agriculture.

Once you click on a dataset, there should be a section to the right of the image that says *Earth Engine Snippet* that starts with `ee.Image` or `ee.ImageCollection`. Copy this snippet.

If the dataset you clicked on has a snippet starting with `ee.ImageCollection`, this indicates that it is a [stack or sequence of images](https://developers.google.com/earth-engine/guides/ic_creating#colab-python). Usually, this means there are images from multiple dates (or regions) that you can choose from. If your Earth Engine snippet starts with `ee.Image` this indicates that there is only one image and you cannot filter by date.

Below, paste the "Earth Engine Snippet" for the dataset you are interested in. For example, for [CFSV2: NCEP Climate Forecast System Version 2, 6-Hourly Products](https://developers.google.com/earth-engine/datasets/catalog/NOAA_CFSV2_FOR6H), it would look like
```
image = ee.ImageCollection("NOAA/CFSV2/FOR6H")
```

In [ ]:
image = 

In the Earth Engine page for your selected dataset, scroll to the very bottom. There should be a **Explore with Earth Engine** section with code snippets showing how to load the data. However, sometimes this code snippet is only available in JavaScript, another programming language. If this is the case, copy and paste the JavaScript code below, and then run the code block, to convert it to Python.

In [ ]:
javascript_code = """
// Paste code below



// Paste code above
"""

geemap.js_snippet_to_py(
    javascript_code, add_new_cell=True, import_ee=True, import_geemap=True, show_map=True
)

The Python code now appears. Check to see if there are any variables with "vis" like visParams. This can also be named specific to the data like "maximumTemperatureVis." If you find it, copy everything inside of the brackets {} and paste it inside of the brackets for `visual_params = {}` in the code below. We will use this later on in the code to visualize the data on a map. If there are no visParams or similar variable, simply put `visual params = {}`

Before running this newly-generated Python code, check to see if there are any `ee.Filter.date()` or mention of dates. If there are, you can replace these with the dates you want. Run the code and you should see a map appear showing the data.

In [ ]:
visual_params = {}

In the Earth Engine catalog, go to the "Bands" page. The first part of the bands page should say **Pixel Size**. Please put the pixel size in the code below, in meters.

In [ ]:
pixel_size = 

On the "Bands" page, scroll down to see the list of bands, and choose the one you are most interested in. Write the name of the band below. For example, the climate forecast dataset has band names like "Temperature_height_above_ground" and "Surface_pressure."

In [ ]:
band_name = ""

## Get data for a specific point

If your dataset is an `ee.ImageCollection`, write the start and end date in the format of "yyyy-mm-dd" like "2025-06-01" (June 1, 2025). Note that the start date is included while the end date is not. For example, the code below will filter the Earth Engine data to images on or after June 1, 2025 and up to, but not including, July 1.

If your dataset is an `ee.Image` feel free to leave the dates blank.
```
start_date = "2025-06-01"
end_date = "2025-07-01"
```

In [ ]:
start_date = ""
end_date = ""

Enter a latitude and longitude for a place you are interested in analyzing. One way to get the latitude and longitude coordinates is to go to [Google Maps](https://www.google.com/maps), left-click on an area you are interested in, and then copy the coordinates.

In [ ]:
latitude = 
longitude = 

For this example, we will use the following details. You can use the same (or different) code.
```
image = ee.ImageCollection("NOAA/CFSV2/FOR6H")

visual_params = {
    "min": 220.0,
    "max": 310.0,
    "palette": ['blue', 'purple', 'cyan', 'green', 'yellow', 'red'],
}
pixel_size = 22264
band_name = "Temperature_height_above_ground"

start_date = "2025-06-01"
end_date = "2025-07-01"

latitude = 25.93805
longitude = -80.78048
```

Double check that the coordinates are correct. If the point is at a random location or over water, it is possible that the latitude and longitude are flipped, so you can revise your code above if needed. In Florida, latitude is generally between 24 and 31, and longitude is between -80 and -88.

In [3]:
point = ee.Geometry.Point(longitude, latitude)

map = folium.Map(location=[latitude, longitude], tiles="OpenStreetMap", zoom_start=10)
folium.Marker(location=[latitude, longitude]).add_to(map)
display(map)

Using the information you found in the first part of this notebook, we'll extract the data at the given date range and coordinates.

For the second line, `.median()` is used for this example, but you can also use `.mean()`, `.sum()`, `.min()`, `.max()`, depending on your data.

In [4]:
data_at_point = (image.filterDate(start_date, end_date)
                      .median()
                      .sample(point, pixel_size)
                      .first()
                      .get(band_name)
                      .getInfo()
)

print(f'From {start_date} to {end_date}, the value for the band "{band_name}" is {data_at_point}.')

From 2025-06-01 to 2025-07-01, the value for the band "Temperature_height_above_ground" is 300.6285095214844.


## Get data over a region

To get the data for a region, you need to load a boundary. For example, this can be a statem, county, or city boundary. For this example, we'll use the Florida state boundary.

We'll load data that we have already downloaded from the [U.S. Census](https://www.census.gov/geographies/mapping-files/time-series/geo/cartographic-boundary.html) for Florida.

In [5]:
fl = gpd.read_file('https://github.com/geo-di-lab/emerge-lessons/raw/refs/heads/main/docs/data/florida_counties.geojson')

Convert the boundary to Earth Engine format.

In [6]:
region = geemap.geopandas_to_ee(fl)

Get the data over the entire region.

In [7]:
data_at_region = (image.filterDate(start_date, end_date)
                      .median()
                      .select(band_name)
                      .clip(region)
)

Show a map of the region.

In [8]:
map = folium.Map(location=[28.263363, -83.497652], tiles="OpenStreetMap", zoom_start=7)
map.add_ee_layer(data_at_region, visual_params, "")
display(map)

Get the average value (of your chosen band) over the entire region.

In [9]:
average_over_region = (data_at_region.reduceRegion(
                                reducer=ee.Reducer.mean(),
                                geometry=region,
                                scale=pixel_size,
                                maxPixels=1e12
                            )
                            .get(band_name)
                            .getInfo()
)

print(f"The average of the band {band_name} for the region from {start_date} to {end_date} is {average_over_region}")

The average of the band Temperature_height_above_ground for the region from 2025-06-01 to 2025-07-01 is 299.62000867726266


This is an example showing how to load data from Google Earth Engine and extract information for a point and a region. More information is available from these resources:
- [Tutorials from the geemap package](https://geemap.org/tutorials/#geemap-tutorials)
- [An Intro to the Earth Engine Python API](https://developers.google.com/earth-engine/tutorials/community/intro-to-python-api)